<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [4]:
import requests

# URL của trang Wikipedia Falcon 9 Launch
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Gửi yêu cầu GET
response = requests.get(url)

# Kiểm tra mã phản hồi HTTP
print("Status code:", response.status_code)

# Xem thử vài dòng đầu của nội dung HTML
print(response.text[:500])


Status code: 200
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


Create a `BeautifulSoup` object from the HTML `response`


In [5]:
from bs4 import BeautifulSoup

# Tạo đối tượng BeautifulSoup từ nội dung HTML
soup = BeautifulSoup(response.text, 'html.parser')

# In thử tiêu đề trang để kiểm tra
print(soup.title)
print(soup.title.string)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
List of Falcon 9 and Falcon Heavy launches - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [6]:
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/Falcon_9'
response = requests.get(url)

# Tạo đối tượng BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# In tiêu đề trang
print(soup.title)
print(soup.title.string)


<title>Falcon 9 - Wikipedia</title>
Falcon 9 - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


In [12]:
from bs4 import BeautifulSoup
import requests

# Gửi yêu cầu đến Wikipedia
url = 'https://en.wikipedia.org/wiki/Falcon_9'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Tìm tất cả bảng trên trang
tables = soup.find_all('table')

print(f"Tổng số bảng tìm được: {len(tables)}")

# Kiểm tra nội dung bảng để xác định đúng bảng cần lấy
for i, table in enumerate(tables):
    print(f"--- Bảng số {i} ---")
    print(table.prettify()[:300])  # In thử 300 ký tự đầu của mỗi bảng
    print("\n")

# Giả sử sau khi kiểm tra, bảng số 2 là bảng cần lấy:
launch_table = tables[2]  # Thay số này nếu kiểm tra thấy khác!

# Tìm dòng đầu tiên có thẻ <tr> và lấy cột từ đó
header_row = launch_table.find('tr')

# Lấy cả thẻ <th> và <td> vì đôi khi Wikipedia dùng <td> cho header
columns = [cell.get_text(strip=True) for cell in header_row.find_all(['th', 'td'])]

print("Tên cột:")
print(columns)


Tổng số bảng tìm được: 24
--- Bảng số 0 ---
<table class="infobox hproduct">
 <caption class="infobox-title">
  Falcon 9
 </caption>
 <tbody>
  <tr>
   <td class="infobox-image" colspan="2">
    <span class="mw-default-size" typeof="mw:File/Frameless">
     <a class="mw-file-description" href="/wiki/File:Falcon_9_logo.svg">
      <img alt="Lo


--- Bảng số 1 ---
<table class="box-Cleanup plainlinks metadata ambox ambox-style ambox-Cleanup" role="presentation">
 <tbody>
  <tr>
   <td class="mbox-image">
    <div class="mbox-image-div">
     <span typeof="mw:File">
      <span>
       <img alt="" class="mw-file-element" data-file-height="48" data-file-width="


--- Bảng số 2 ---
<table class="wikitable">
 <tbody>
  <tr>
   <td>
    Height
   </td>
   <td>
    41.2 m / 135.2 ft
   </td>
  </tr>
  <tr>
   <td>
    Height (with interstage)
   </td>
   <td>
    47.7 m / 156.5 ft
   </td>
  </tr>
  <tr>
   <td>
    Diameter
   </td>
   <td>
    3.7 m / 12 ft
   </td>
  </tr>
  <


--- Bảng số

Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


Starting from the third table is our target table contains the actual launch records.


In [15]:
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/Falcon_9'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Lấy tất cả bảng trên trang
html_tables = soup.find_all('table')

print(f"Số bảng tìm được: {len(html_tables)}")

# Lấy bảng thứ 3 (đếm từ 0)
first_launch_table = html_tables[2]
print(first_launch_table.prettify()[:500])  # In thử 500 ký tự đầu tiên để kiểm tra


Số bảng tìm được: 24
<table class="wikitable">
 <tbody>
  <tr>
   <td>
    Height
   </td>
   <td>
    41.2 m / 135.2 ft
   </td>
  </tr>
  <tr>
   <td>
    Height (with interstage)
   </td>
   <td>
    47.7 m / 156.5 ft
   </td>
  </tr>
  <tr>
   <td>
    Diameter
   </td>
   <td>
    3.7 m / 12 ft
   </td>
  </tr>
  <tr>
   <td>
    Empty mass
   </td>
   <td>
    25,600 kg / 56,423 lb
   </td>
  </tr>
  <tr>
   <td>
    Propellant mass
   </td>
   <td>
    395,700 kg/ 872,369 lb
   </td>
  </tr>
  <tr>
   <td>
  


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [18]:
from bs4 import BeautifulSoup
import requests

def extract_column_from_header(th):
    """Hàm kiểm tra và trả về tên cột nếu tồn tại."""
    name = th.get_text(strip=True)
    return name if name else None

# Gửi request lấy HTML
url = 'https://en.wikipedia.org/wiki/Falcon_9'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Tìm tất cả bảng
html_tables = soup.find_all('table')

# Chọn bảng thứ 3 (đếm từ 0 là bảng số 2)
first_launch_table = html_tables[2]

# Lấy tất cả th trong bảng đó
ths = first_launch_table.find_all('th')

# Lấy tên cột
column_names = []
for th in ths:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)





Check the extracted column names


In [19]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Lấy HTML từ Wikipedia
url = 'https://en.wikipedia.org/wiki/Falcon_9'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Tìm tất cả các bảng trên trang
html_tables = soup.find_all('table', {'class': 'wikitable'})

# Chọn bảng phù hợp: thông thường là bảng thứ 3 (chỉnh số nếu cần)
launch_table = html_tables[2]

# Đọc bảng HTML trực tiếp với pandas
df = pd.read_html(str(launch_table))[0]

# In thử vài dòng đầu tiên để kiểm tra
print(df.head())


           Version        v1.0 (retired)        v1.1 (retired)  \
           Version        v1.0 (retired)        v1.1 (retired)   
0  Stage 1 engines         9 × Merlin 1C         9 × Merlin 1D   
1  Stage 2 engines  1 × Merlin 1C Vacuum  1 × Merlin 1D Vacuum   
2  Max. height (m)               53[114]               68.4[3]   
3     Diameter (m)             3.66[115]             3.66[116]   
4   Initial thrust   3.807 MN (388.2 tf)    5.9 MN (600 tf)[3]   

                             Full Thrust[8]  \
              Block 3 and Block 4 (retired)   
0             9 × Merlin 1D (upgraded)[113]   
1  1 × Merlin 1D Vacuum (upgraded)[96][113]   
2                                 70[2][96]   
3                                  3.66[96]   
4                6.804 MN (693.8 tf)[2][96]   

                                                      
                          Block 5 (active)[111][112]  
0                           9 × Merlin 1D (upgraded)  
1  1 × Merlin 1D Vacuum (upgraded) (short 

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://en.wikipedia.org/wiki/Falcon_9'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Lấy tất cả các bảng wikitable
html_tables = soup.find_all('table', {'class': 'wikitable'})

# Chọn bảng thứ 3
first_launch_table = html_tables[2]

# Tạo danh sách chứa dữ liệu từ các hàng
launch_dict = {
    "Date": [],
    "Time": [],
    "Booster_Version": [],
    "Launch_Site": [],
    "Payload": [],
    "Payload_Mass": [],
    "Orbit": [],
    "Customer": [],
    "Mission_Outcome": [],
    "Landing_Outcome": []
}

# Duyệt từng hàng (bỏ hàng đầu vì là header)
for row in first_launch_table.find_all('tr')[1:]:
    cells = row.find_all(['th', 'td'])

    if len(cells) >= 9:  # Kiểm tra đủ cột cần thiết, số này có thể điều chỉnh tùy bảng
        # Loại bỏ tham chiếu trong text như [1], [a]
        cell_texts = [cell.get_text(strip=True).split('[')[0].replace('\xa0', ' ') for cell in cells]

        # Thêm dữ liệu vào launch_dict
        launch_dict["Date"].append(cell_texts[0] if len(cell_texts) > 0 else None)
        launch_dict["Time"].append(cell_texts[1] if len(cell_texts) > 1 else None)
        launch_dict["Booster_Version"].append(cell_texts[2] if len(cell_texts) > 2 else None)
        launch_dict["Launch_Site"].append(cell_texts[3] if len(cell_texts) > 3 else None)
        launch_dict["Payload"].append(cell_texts[4] if len(cell_texts) > 4 else None)
        launch_dict["Payload_Mass"].append(cell_texts[5] if len(cell_texts) > 5 else None)
        launch_dict["Orbit"].append(cell_texts[6] if len(cell_texts) > 6 else None)
        launch_dict["Customer"].append(cell_texts[7] if len(cell_texts) > 7 else None)
        launch_dict["Mission_Outcome"].append(cell_texts[8] if len(cell_texts) > 8 else None)
        launch_dict["Landing_Outcome"].append(cell_texts[9] if len(cell_texts) > 9 else None)

# Chuyển thành DataFrame
df_launches = pd.DataFrame(launch_dict)

# Xem thử kết quả
print(df_launches.head())


Empty DataFrame
Columns: [Date, Time, Booster_Version, Launch_Site, Payload, Payload_Mass, Orbit, Customer, Mission_Outcome, Landing_Outcome]
Index: []


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
